# 🎯 Hull Tactical Market Prediction - Simple Pipeline

**3개의 스크립트만 실행하면 끝!**

## 📌 Setup Instructions

### Step 1: Upload Files to Kaggle Dataset

1. **Go to**: https://www.kaggle.com/datasets
2. **Click**: "New Dataset"
3. **Upload ALL files**:
   - All `.py` files from `src/` and `scripts/`
   - `params.yaml` from `conf/`
4. **Name it**: `mydata`
5. **Click**: "Create"

### Step 2: Configure Dataset Name

셀 3에서 데이터셋 이름 확인:

```python
DATASET_NAME = "mydata"  # 여기 확인!
```

### Step 3: Run Notebook

1. **Accelerator**: None (CPU로 충분합니다)
2. **Click**: "Run All"
3. **Wait**: ~2-3 minutes
4. **Download**: `submissions/submission.parquet`
5. **Submit**: Upload to competition

## 🚀 Pipeline:

1. **optimize_return_model.py** (~1 min)
   - Feature engineering + selection
   - Train LightGBM return model (5-fold CV)

2. **optimize_risk_model.py** (~1 min)
   - Create risk labels
   - Train LightGBM risk model (5-fold CV)

3. **optimize_position_strategy.py** (~30 sec)
   - Optimize position mapping
   - Generate submission.parquet

**Total: ~2-3 minutes** ⚡

Good luck! 🚀

## 1️⃣ Setup: Configure Dataset Path

In [ ]:
import os
import sys
from pathlib import Path
import yaml

# ========== CONFIGURATION: 데이터셋 이름 (여기만 수정하세요!) ==========
DATASET_NAME = "mydata"
# ====================================================================

print("="*80)
print("SETTING UP PATHS")
print("="*80)

# Kaggle 데이터셋 경로
DATASET_PATH = Path(f"/kaggle/input/{DATASET_NAME}")

if DATASET_PATH.exists():
    sys.path.insert(0, str(DATASET_PATH))
    print(f"✓ Dataset found: {DATASET_PATH}")
    
    # ========== src 모듈 가상 생성 ==========
    import importlib.util
    import types
    
    src_module = types.ModuleType('src')
    src_module.__path__ = [str(DATASET_PATH)]
    sys.modules['src'] = src_module
    
    # 의존성 순서대로 모듈 로드
    py_files_to_load = [
        'utils', 'metric', 'cv', 'data', 'features', 'models', 
        'tuner', 'backtest', 'position', 'risk', 'interpretability', 
        'ensemble', 'timeseries_risk'
    ]
    
    for module_name in py_files_to_load:
        py_file = DATASET_PATH / f"{module_name}.py"
        if py_file.exists():
            full_module_name = f"src.{module_name}"
            spec = importlib.util.spec_from_file_location(full_module_name, py_file)
            if spec and spec.loader:
                module = importlib.util.module_from_spec(spec)
                sys.modules[full_module_name] = module
                setattr(src_module, module_name, module)
                try:
                    spec.loader.exec_module(module)
                    print(f"  ✓ Loaded: src.{module_name}")
                except Exception as e:
                    print(f"  ⚠️  Error loading src.{module_name}: {e}")
    
    print(f"\n✓ Created virtual 'src' module")
    
    # Working directory 설정
    os.chdir("/kaggle/working")
    print(f"✓ Working directory: {os.getcwd()}")
    
else:
    print(f"❌ Dataset not found: {DATASET_PATH}")
    input_dir = Path("/kaggle/input/")
    if input_dir.exists():
        for item in input_dir.iterdir():
            print(f"  📁 {item.name}")
    raise FileNotFoundError(f"Dataset '{DATASET_NAME}' not found!")

# ========== CONFIG 복사 ==========
config_path = DATASET_PATH / "params.yaml"
if config_path.exists():
    working_config_dir = Path("/kaggle/working/conf")
    working_config_dir.mkdir(exist_ok=True)
    
    import shutil
    shutil.copy(config_path, working_config_dir / "params.yaml")
    
    # conf 디렉토리를 Python path에 추가
    sys.path.insert(0, str(working_config_dir.parent))
    print(f"\n✓ Config copied to: {working_config_dir}/params.yaml")
else:
    print(f"\n⚠️  Config file not found: {config_path}")

print("\n✅ Setup complete!")

## 2️⃣ Script 1: Train Return Model (~10 min)

In [ ]:
print("="*80)
print("STEP 1/3: TRAINING RETURN MODEL")
print("="*80)

import shutil

# Copy data files
data_dir = Path("/kaggle/working/data/raw")
data_dir.mkdir(parents=True, exist_ok=True)

train_src = Path("/kaggle/input/hull-tactical-market-prediction/train.csv")
test_src = Path("/kaggle/input/hull-tactical-market-prediction/test.csv")

if train_src.exists():
    shutil.copy(train_src, data_dir / "train.csv")
    print(f"✓ Copied train.csv")

if test_src.exists():
    shutil.copy(test_src, data_dir / "test.csv")
    print(f"✓ Copied test.csv")

# Run optimize_return_model.py
script_path = DATASET_PATH / "optimize_return_model.py"

if script_path.exists():
    print(f"\n▶ Running: {script_path.name}\n")
    %run {script_path}
    print("\n✅ Return model training complete!")
else:
    raise FileNotFoundError(f"Script not found: {script_path}")

print("\n📊 Results saved to: artifacts/models_optimized/")

## 3️⃣ Script 2: Train Risk Model (~10 min)

In [ ]:
print("="*80)
print("STEP 2/3: TRAINING RISK MODEL")
print("="*80)

script_path = DATASET_PATH / "optimize_risk_model.py"

if script_path.exists():
    print(f"▶ Running: {script_path.name}\n")
    %run {script_path}
    print("\n✅ Risk model training complete!")
else:
    raise FileNotFoundError(f"Script not found: {script_path}")

print("\n📊 Results saved to: artifacts/models_risk_optimized/")

## 4️⃣ Script 3: Optimize Position Strategy + Generate Submission (~10 min)

In [ ]:
print("="*80)
print("STEP 3/3: GENERATING SUBMISSION")
print("="*80)

script_path = DATASET_PATH / "optimize_position_strategy.py"

if script_path.exists():
    print(f"▶ Running: {script_path.name}\n")
    %run {script_path}
    print("\n✅ Submission generation complete!")
else:
    raise FileNotFoundError(f"Script not found: {script_path}")

# Verify submission
submission_path = Path("submissions/submission.parquet")
if submission_path.exists():
    import pandas as pd
    submission = pd.read_parquet(submission_path)
    print(f"\n✅ Submission verified:")
    print(f"  - Rows: {len(submission):,}")
    print(f"  - Allocation range: [{submission['allocation'].min():.4f}, {submission['allocation'].max():.4f}]")
    print(f"  - Mean allocation: {submission['allocation'].mean():.4f}")
    print(f"\n📥 Download 'submissions/submission.parquet' and submit!")
else:
    print(f"\n❌ Submission file not found: {submission_path}")

## 📊 Pipeline Summary

### Workflow:
```
Setup (Cell 3) → Return Model (Cell 4) → Risk Model (Cell 5) → Submission (Cell 6)
```

**Total Time: ~2-3 minutes** (데이터가 작아서 CPU로 충분합니다)

### 🎯 Ready!

1. ✅ Upload files to Kaggle Dataset (`mydata`)
2. ✅ Set Accelerator to **None** (CPU)
3. ✅ Click "Run All"
4. ✅ Download `submissions/submission.parquet`
5. ✅ Submit!

**Good luck! 🚀**